In [240]:
import numpy as np
import scipy.linalg as la
import time
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sp
import threading
import concurrent.futures
import networkx as nx
from mpl_toolkits import mplot3d
from scipy import sparse as spa
from scipy.sparse import linalg as sla
from collections import defaultdict
from matplotlib.colors import LogNorm

%matplotlib inline
sp.set_printoptions(precision=4, suppress=True)
np.set_printoptions(precision=4)


In [2]:
def read_data(filename):

    print('reading input data')
    df = pd.read_csv(filename, sep=';')
    df.drop_duplicates()

    return df


In [3]:
def file_to_graph(filename='edges.csv'):

    """Read data file and return graph. Target nodes are stored in a list as keys of (empty) dictionaries.
    e.g. print(G[236])  {186: {}, 84: {}, 62: {},...}
    print(type(G[236])) (<class 'networkx.classes.coreviews.AtlasView'>)"""
    print('reading input data')
    df = read_data(filename)
    # A_ij means j points to i
    graph = nx.from_pandas_edgelist(df, source='src', target='trg')

    return graph


In [4]:
def graph_to_sparse_matrix(graph):

    """Converts the graph to a sparse matrix in coo format, reconvert if necessary"""
    sparse = nx.to_scipy_sparse_matrix(graph)
    sparse = sparse.tocsr()

    return sparse


In [5]:
def edgelist_to_sparse_matrix(filename='edges.csv'):

    """Read data file and return sparse matrix in coordinate format."""
    print('converting edge list to sparse matrix')
    df = read_data(filename)
    # A_ij means j points to i
    rows = df['trg']  # Not a copy, just a reference.
    cols = df['src']
    ones = np.ones(len(rows), np.uint32)
    sparse = spa.coo_matrix((ones, (rows, cols)))
    # print(sparse.shape())
    sparse = sparse.tocsr()

    # print(sparse.shape())
    return sparse


In [6]:
def graph_to_adjacency_dicts(graph):

    """Converts the graph to a dictionary representation of every node and it's adjacent nodes"""
    adj = nx.to_dict_of_lists(graph)

    return adj


In [7]:
def get_google_matrix(graph_in):

    print('creating google mmatrix')

    adj = graph_to_adjacency_dicts(graph_in)
    # get all indices for nonzero elements in a sparse matrix
    # [((i, j), a[i, j]) for i, j in zip(*a.nonzero())]

    # S = nx.Graph()
    graph = nx.Graph()
    num_nodes = len(adj)
    for node in adj:
        graph.add_node(node)
    alpha = 0.85
    for count, j in enumerate(adj):
        print(count)
        out_edges = len(adj[j])
        k_out = out_edges if out_edges > 0 else 1 / num_nodes
        for i in adj:
            adj_ij = 1 if j in adj[i] else 0
            weight = alpha * adj_ij / k_out + (1 - alpha) / num_nodes
            graph.add_edge(i, j, weight=weight)
            # the next three lines are equivalent to the previous and can be used if S_ij is needed elsewhere
            # S[i][j]['weight'] = A_ij / k_out
            # S_ij = S[i][j]['weight']
            # G[i][j]['weight'] = alpha * S_ij + (1 - alpha) / N
    nx.write_adjlist(graph, "graph.adjlist")
    G = graph_to_sparse_matrix(graph)

    return G


In [27]:
def get_eigen(g):

    print('calculating the eigenvalues')
    l_ev, v_l, v_r = la.eig(g, left=True)
    # l_ev vector of eigenvalues
    # v_l left eigenvector
    # v_r right eigenvector

    # usage if only interested in the largest eigenvalue:
    v_r = v_r[:, 0].real
    v_l = v_l[:, 0].real
    l_c = l_ev[0].real

    return l_c, v_l, v_r


In [241]:
def get_eigen_sparse(g):
    eig_r, v_r = sla.eigs(g, k=1, which='LR')
    eig_l, v_l = sla.eigs(g.transpose(), k=1, which='LR')
    
    if not eig_l == eig_r:
        print('the eigenvalues are different!')
    
    return eig_r, v_l, v_r


In [224]:
a = spa.random(5, 5, density=0.25)
print(a.todense())
print(a.transpose().todense())

[[0.8096 0.     0.     0.9148 0.    ]
 [0.9038 0.5422 0.     0.     0.    ]
 [0.     0.     0.     0.     0.    ]
 [0.     0.     0.6345 0.3459 0.    ]
 [0.     0.     0.     0.     0.    ]]
[[0.8096 0.9038 0.     0.     0.    ]
 [0.     0.5422 0.     0.     0.    ]
 [0.     0.     0.     0.6345 0.    ]
 [0.9148 0.     0.     0.3459 0.    ]
 [0.     0.     0.     0.     0.    ]]


In [233]:
eig_r, v_r = sla.eigs(a, k=1, which='LR')
eig_l, v_l = sla.eigs(a.transpose(), k=1, which='LR')
print('eigenvalues right: \n', eig_r.real, '\neigenvalues left: \n', eig_l.real, '\nright eigenvectors: \n', v_r.real, '\nleft eigenvectors: \n', v_l.real)

print('A * v_r = \n', a @ v_r[:, 0].real)
print('eig * v_r = \n', eig_r[0].real * v_r[:, 0].real)

print('v_l * A = \n', v_l[:, 0].real @ a)
print('eig * v_l = \n', eig_r[0].real * v_l[:, 0].real)

eig, v_l, v_r = la.eig(a.todense(), left=True)
print('\neigenvalues: \n', eig.real, '\nright eigenvectors: \n', v_r.real, '\nleft eigenvectors: \n', v_l.real)

print('A * v_r = \n', a @ v_r[:, 1].real)
print('eig * v_r = \n', eig[1].real * v_r[:, 1].real)

print('v_l * A = \n', v_l[:, 1].real @ a)
print('eig * v_l = \n', eig[1].real * v_l[:, 0].real)

eig_r, v_r = np.linalg.eig(a.todense())
eig_l, v_l = np.linalg.eig(a.transpose().todense())
print('\neigenvalues right: \n', eig_r.real, '\neigenvalues left: \n', eig_l.real, '\nright eigenvectors: \n', v_r.real, '\nleft eigenvectors: \n', v_l.real)

print('A * v_r = \n', a @ v_r[:, 1].real)
print('eig * v_r = \n', eig_r[1].real * v_r[:, 1].real)

print('v_l * A = \n', v_l[:, 1].real.transpose() @ a)
print('eig * v_l = \n', v_l[:, 1].real * eig_l[1].real)


eigenvalues right: 
 [0.8096] 
eigenvalues left: 
 [0.8096] 
right eigenvectors: 
 [[0.2837]
 [0.9589]
 [0.    ]
 [0.    ]
 [0.    ]] 
left eigenvectors: 
 [[ 0.3706]
 [-0.    ]
 [ 0.5729]
 [ 0.731 ]
 [ 0.    ]]
A * v_r = 
 [0.2297 0.7763 0.     0.     0.    ]
eig * v_r = 
 [0.2297 0.7763 0.     0.     0.    ]
v_l * A = 
 [ 0.3    -0.      0.4639  0.5919  0.    ]
eig * v_l = 
 [ 0.3    -0.      0.4639  0.5919  0.    ]

eigenvalues: 
 [0.5422 0.8096 0.3459 0.     0.    ] 
right eigenvectors: 
 [[ 0.      0.2837 -0.211   0.4567  0.    ]
 [ 1.      0.9589  0.9716 -0.7613  0.    ]
 [ 0.      0.      0.      0.2203  0.    ]
 [ 0.      0.      0.107  -0.4042  0.    ]
 [ 0.      0.      0.      0.      1.    ]] 
left eigenvectors: 
 [[-0.1379  0.3706  0.      0.      0.    ]
 [ 0.0408  0.      0.      0.      0.    ]
 [-0.7523  0.5729  0.878   1.      0.    ]
 [-0.6429  0.731   0.4786  0.      0.    ]
 [ 0.      0.      0.      0.      1.    ]]
A * v_r = 
 [0.2297 0.7763 0.     0.     0.    ]

In [9]:
def decompose_matrix(g, count):

    g_rr = g[0:count, 0:count]
    g_rs = g[0:count, count:]
    g_sr = g[count:, 0:count]
    g_ss = g[count:, count:]

    return g_rr, g_rs, g_sr, g_ss


In [10]:
def calc_g_pr(g_rs, g_sr, p_c, l, v_r, v_l):

    print('calculating G_pr')
    # the following two lines give equivalent results, might be different in performance
    # print('G_rs ', G_rs, '\n p_c:\n', P_c, '\n G_sr:\n', G_sr)
    # G_pr = (G_rs @ p_c @ G_sr)/(1-l)
    # print('G_rs ', G_rs, '\n v_r:\n', v_r, '\n G_sr:\n', G_sr, '\n v_l:\n', v_l)
    vr_x = (g_rs @ v_r)
    vl_x = (v_l @ g_sr)
    g_pr = (np.outer(vr_x, vl_x) / (1 - l)).real
    # print('\nvr_x:\n', vr_x, '\nvl_x:\n', vl_x, '\ng_pr:\n', g_pr)

    return g_pr


In [11]:
def calc_sum_expr(g_ss_x, size):

    sum_expr = np.zeros((size, size))
    epsilon = np.ones((size, size)) * 0.001
    sum_expr += g_ss_x.power(0)
    # print('sum_expr: \n', sum_expr, '\nprev:\n', prev, '\nepsilon:\n', epsilon)
    # print('\nsum_expr - prev:\n', np.abs(sum_expr - prev))
    # print((np.abs(sum_expr - prev) > epsilon).all())
    l = 1
    while ((np.linalg.matrix_power(g_ss_x, l)) > epsilon).any() & (g_ss_x.power(l) < np.inf).all():
        # print('\nsum_expr - prev:\n', np.abs(sum_expr - prev))
        # print('\nsum_expr:\n', sum_expr)
        sum_expr += np.linalg.matrix_power(g_ss_x, l)
        #  'sum_expr: ', sum_expr, '\nprev:\n', prev,
        # print('\nsum_expr:\n', sum_expr)
        l += 1

    return sum_expr


In [12]:
def calc_g_qr(q_c, g_ss, g_rs, g_sr, size):

    print('calculating G_qr')
    g_ss_x = q_c * g_ss * q_c
    # print('\ng_ss:\n', g_ss, '\nq_c:\n', q_c, '\ng_ss_x:\n', g_ss_x)
    # print('g_ss_x: \n', g_ss_x, '\nmatrix_power(g_ss_x, 0):\n', np.linalg.matrix_power(g_ss_x, 0))
    sum_expr = calc_sum_expr(g_ss_x, size)
    sum_expr_g_sr = sum_expr @ g_sr
    prod = q_c * sum_expr_g_sr

    g_qr = g_rs @ prod

    return g_qr


In [13]:
def calc_sum_expr_g_sj(sum_expr, col):

    sum_expr_g_sr_col = sum_expr @ col

    return sum_expr_g_sr_col


In [245]:
def calc_g_r(g, count, size):

    print('calculating G_r')
    g_rr, g_rs, g_sr, g_ss = decompose_matrix(g, count)
    # print('\ng:\n', g, '\ng_rr:\n', g_rr, '\ng_rs:\n', g_rs, '\ng_sr:\n', g_sr, '\ng_ss:\n', g_ss)
    # l_c, v_l, v_r = get_eigen(g_ss.todense())
    l_c, v_l, v_r = get_eigen_sparse(g_ss)
    
    p_c = np.dot(v_r, v_l).real
    q_c = 1 - p_c

    g_pr = calc_g_pr(g_rs, g_sr, p_c, l_c, v_l, v_r)
    g_qr = calc_g_qr(q_c, g_ss, g_rs, g_sr, size - count)
    # print('\ng_rr: \n', g_rr, '\ng_pr: \n', g_pr, '\ng_qr: \n', g_qr)
    g_r = g_rr + g_pr + g_qr

    return g_r


In [237]:
def main():

    # graph = file_to_graph('edges.csv')
    # print('a: \n', a)

    # Creating the graph is necessary for the first time only, can be read from the files otherwise
    # g = get_google_matrix(graph)
    g = nx.read_adjlist("graph.adjlist")
    print('finished reading the graph')
    # sparse = edgelist_to_sparse_matrix('edges.csv')
    # spa.save_npz('sparse_matrix.npz', sparse)
    # print('the graph has been saved to sparse_matrix.npz')
    sparse = spa.load_npz("sparse_matrix.npz")
    print('finished reading the matrix')
    size = len(g)
    count = 50

    start_time = time.time()
    print('\nStarting the code for a matrix of size {0} and count of {1}'.format(size, count))
    g_r = calc_g_r(sparse, count, size)
    # print('\ng_r:\n', g_r)
    plt.matshow(g_r)
    plt.title('size = {0}, count = {1}\n'.format(size, count))
    plt.show()
    plt.colorbar()
    stop_time = time.time()
    running_time = stop_time - start_time
    print('\nRunning the code for a graph of size {0} and count of {1} took {2} seconds'
          .format(size, count, running_time))


In [243]:
main()

MemoryError: 

In [247]:
a = [1, 2, 2]
b = [2, 2, 1]
c = np.dot(a, b)
c

8